# QCNN demonstration with Noisy Devices

## 1. Load Noisemodel with IBMQ

Noise Model (Fake Provider)

In [ ]:
n_qubits = 4

: 

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.aer.backends import AerSimulator
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel

IBMQ.load_account()
noisy = FakeJakarta()

noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=n_qubits,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

Real device

In [ ]:
TOKEN_tak = "798c4f141e8fa071c14823a956f816c8dda622f9f00b95cae62cc5b1a3105b4fe4845a234d661f20281f6b77ac0299e3c4367f6a3b58db16c36bcde3a9cb1151"
TOKEN_dkp = "3ae7ac10f40eb88c7ebb0eca20aa0788e7a96da729e2c3848d9864684362aaf50dfdda7cb3ec1ffd75eb7ed5b44f7c14f2f17419cf600ce14437ee7cd00ac75b"

IBMQ.save_account(token=TOKEN_dkp, overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-skku')
backend = "ibmq_jakarta"

dev_jakarta = qml.device(
    'qiskit.ibmq',
    wires=4,
    shots=1024,
    backend=backend,
    provider=provider
)

## 2. Feature Mapping Circuits

Create Feature Mapping Circuits. 

Note: This is 4 qubit feature mapping circuits used for demonstration.

Unlike noiseless simulation, last to first qubit CNOT gates are omitted for connectivity.

Also number of layers is set to 1 (N_layers = 3 for noiseless simulation).

In [ ]:
import embedding

def Noisy_Four_QuantumEmbedding1(input):
    for j in range(4):
        qml.Hadamard(wires=j)
        embedding.exp_Z(input[j], wires=j)
    for k in range(3):
        embedding.exp_ZZ2(input[k], input[k+1], wires=[k,k+1])
    #exp_ZZ2(input[3], input[0], wires=[3, 0])                  Removed for connectivity

def Noisy_Four_QuantumEmbedding1_inverse(input):
    #exp_ZZ2(input[3], input[0], wires=[3, 0], inverse=True)    Removce for connectivity
    for k in reversed(range(3)):
        embedding.exp_ZZ2(input[k], input[k+1], wires=[k,k+1], inverse=True)
        qml.Barrier()
    for j in range(4):
        embedding.exp_Z(input[j], wires=j, inverse=True)
        qml.Hadamard(wires=j)

# Quantum Embedding 2 for model 2
def Noisy_Four_QuantumEmbedding2(input):
    for j in range(4):
        qml.Hadamard(wires=j)
        embedding.exp_Z(input[j], wires=j)
    for k in range(3):
        embedding.exp_ZZ1(input[4+k], wires=[k, k+1])
    #exp_ZZ1(input[15], wires=[7,0])                        Removed for connectivity

def Noisy_Four_QuantumEmbedding2_inverse(input):
    #embedding.exp_ZZ1(input[15], wires=[7,0], inverse=True) Removed for connectivity
    for k in reversed(range(3)):
        embedding.exp_ZZ1(input[k+4], wires=[k,k+1], inverse=True)
    qml.Barrier()
    for j in range(4):
        embedding.exp_Z(input[j], wires=j, inverse=True)
        qml.Hadamard(wires=j)

In [ ]:
import torch
from torch import nn

# Hybrid Model 1
@qml.qnode(dev_fake, interface="torch")
def circuit1(inputs): 
    Noisy_Four_QuantumEmbedding1(inputs[0:4])
    Noisy_Four_QuantumEmbedding1_inverse(inputs[4:8])
    return qml.probs(wires=range(4))

class Noisy_Model1_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1 = qml.qnn.TorchLayer(circuit1, weight_shapes={})
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(4, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 4)
        )
    def forward(self, x1, x2):
        x1 = self.linear_relu_stack1(x1)
        x2 = self.linear_relu_stack1(x2)
        x = torch.concat([x1, x2], 1)
        x = self.qlayer1(x)
        return x[:,0]

# Hybrid Model 2
@qml.qnode(dev_fake, interface="torch")
def circuit2(inputs): 
    Noisy_Four_QuantumEmbedding2(inputs[0:7])
    Noisy_Four_QuantumEmbedding2_inverse(inputs[7:14])
    return qml.probs(wires=range(4))

class Noisy_Model2_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer2 = qml.qnn.TorchLayer(circuit2, weight_shapes={})
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
    def forward(self, x1, x2):
        x1 = self.linear_relu_stack2(x1)
        x2 = self.linear_relu_stack2(x2)
        x = torch.concat([x1, x2], 1)
        x = self.qlayer2(x)
        return x[:,0]

In [ ]:
print("Circuit1: \n")
print(
    qml.draw(
        circuit1,
        expansion_strategy="device",
        show_matrices=False
    )([1,2,3,4,1,2,3,4])
)

print("Circuit2: \n")
print(
    qml.draw(
        circuit2,
        expansion_strategy="device",
        show_matrices=False
    )([1,2,3,4,5,6,7,1,2,3,4,5,6,7])
)

# 4. Load Datasets

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import data
import training


feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

N_valid, N_test = 500, 1000
X1_new_valid, X2_new_valid, Y_new_valid = training.new_data(N_valid, X_test, Y_test)
X1_new_test, X2_new_test, Y_new_test = training.new_data(N_test, X_test, Y_test)

# 5. Train Models

In [ ]:
def train_models(model_name):
    train_loss = []
    if model_name == 'Model1_Fidelity':
        model = Model1_Fidelity()
        PATH = '/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/Analysis/Model1_Fidelity.pt'
    model.train()

    loss_fn = torch.nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.01)
    for it in range(200):
        X1_batch, X2_batch, Y_batch = new_data(10, X_train, Y_train)
        X1_batch, X2_batch, Y_batch = X1_batch, X2_batch, Y_batch

        pred = model(X1_batch, X2_batch)
        pred, Y_batch = pred.to(torch.float32), Y_batch.to(torch.float32)
        loss = loss_fn(pred, Y_batch)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()
        opt.step()

        print(f"Iterations: {it} Loss: {loss.item()}")
    
    torch.save(model.state_dict(), PATH)


#train_models('Model1_Fidelity')
train_models('Model1_Fidelity')

In [ ]:
PATH = '/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/Analysis/Model1_Fidelity.pt'
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test), torch.tensor(X0_test)

X1_train, X0_train = [], []
for i in range(len(X_train)):
    if Y_train[i] == 1:
        X1_train.append(X_train[i])
    else:
        X0_train.append(X_train[i])
X1_train, X0_train = torch.tensor(X1_train), torch.tensor(X0_train)


dev = qml.device('default.qubit', wires=4)
@qml.qnode(dev, interface="torch")
def Four_Distance(inputs): 
    Four_QuantumEmbedding(inputs[0:4])
    return qml.density_matrix(wires=range(4))

class Distances(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1_distance = qml.qnn.TorchLayer(Four_Distance, weight_shapes={})
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(4, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 4)
        )
    def forward(self, x1, x0, Distance, Trained):
        if Trained:
            x1 = self.linear_relu_stack1(x1)
            x0 = self.linear_relu_stack1(x0)
        rhos1 = self.qlayer1_distance(x1)
        rhos0 = self.qlayer1_distance(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0
        if Distance == 'Trace':
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))

Model1_Fidelity_Distance = Distances()
Model1_Fidelity_Distance.load_state_dict(torch.load(PATH))

In [ ]:
# Distances Before Training
Trace_before_traindata = Model1_Fidelity_Distance(X1_train, X0_train, 'Trace', False)
Trace_before_testdata = Model1_Fidelity_Distance(X1_test, X0_test, 'Trace', False)
print(f"Trace Distance (Training Data) Before: {Trace_before_traindata}")
print(f"Trace Distance (Test Data) Before: {Trace_before_testdata}")

# Distances After training with Model1_Fidelity
Trace_Fidelity_traindata = Model1_Fidelity_Distance(X1_train, X0_train, 'Trace', True)
Trace_Fidelity_testdata = Model1_Fidelity_Distance(X1_test, X0_test, 'Trace', True)
print(f"Trace Distance (Training Data) After Model1 Fidelity: {Trace_Fidelity_traindata}")
print(f"Trace Distance (Test Data) After Model1 Fidelity: {Trace_Fidelity_testdata}")

## 6. Training QCNN

In [ ]:
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
class x_transform(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(4, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 4)
        )
    def forward(self, x):
        x = self.linear_relu_stack1(x)
        return x.detach().numpy()

model = x_transform()
model.load_state_dict(torch.load(PATH))
#model.load_state_dict(torch.load(PATH_Model1_HSinner, map_location=device))

In [ ]:
def statepreparation(x, Trained):
    if Trained:
        x = model(torch.tensor(x))
    Four_QuantumEmbedding(x)

def SU_4(params, wires): # 15 params
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])

def U_TTN(params, wires):  # 2 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

def QCNN_four(params):
    param1 = params[0:2]
    param2 = params[2:4]
    U_TTN(param1, wires=[0, 1])
    U_TTN(param1, wires=[2, 3])
    U_TTN(param1, wires=[1, 2])
    U_TTN(param1, wires=[3, 0])
    U_TTN(param2, wires=[0, 2])


@qml.qnode(dev_fake)
def QCNN_classifier(params, x, Trained):
    statepreparation(x, Trained)
    qml.Barrier()
    QCNN_four(params)
    return qml.expval(qml.PauliZ(2))

def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)

def cost(weights, X_batch, Y_batch, Trained):
    preds = [QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)

steps = 400
learning_rate = 0.01
batch_size = 10
def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

In [ ]:
print(
    qml.draw(
        QCNN_classifier,
        expansion_strategy="device",
        show_matrices=False
    )(x=torch.ones(4), params=np.ones(30), Trained=True)
)

In [ ]:
loss_not_trained, weights_not_trained = circuit_training(X_train, Y_train, False)
loss_trained, weights_trained = circuit_training(X_train, Y_train, True)

In [ ]:
LB_before_traindata = 0.5 * (1 - 0.3409276604652405)
LB_Fidelity_traindata = 0.5 * (1 - 0.9015634655952454)

import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 4)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(loss_not_trained)), loss_not_trained, label="Without Pre-training", c=clrs[0])

    ax.plot(range(len(loss_trained)), loss_trained, label="With Pre-training", c=clrs[1])

    ax.plot(range(400), np.ones(400) * LB_before_traindata, label="Lower Bound without Pre-training", c=clrs[2])
    ax.plot(range(400), np.ones(400) * LB_Fidelity_traindata, label="Lower Bound with Pre-training", c=clrs[3])

ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("QCNN Loss History Trained with Model1 Fidelity")
ax.legend()

In [ ]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


predictions_trained = [QCNN_classifier(weights_trained, x, Trained=True) for x in X_test]
predictions_not_trained = [QCNN_classifier(weights_not_trained, x, Trained=False) for x in X_test]

accuracy_trained = accuracy_test(predictions_trained, Y_test)
accuracy_not_trained = accuracy_test(predictions_not_trained, Y_test)

In [ ]:
print(f" accuracty after pre-training: {accuracy_trained}")
print(f" accuracty without pre-training: {accuracy_not_trained}")